In [1]:
!pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client --quiet

In [2]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [3]:
# 1. SETUP GMAIL AUTHENTICATION
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None
    if os.path.exists('contents/token.json'):
        creds = Credentials.from_authorized_user_file('contents/token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('contents/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0, open_browser=False) # Modified to not open browser
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()

In [4]:
# Checked if Gmail is connected
profile = service.users().getProfile(userId='me').execute()
print("Email:", profile["emailAddress"])
print("Total messages:", profile["messagesTotal"])

Email: forsignins043@gmail.com
Total messages: 236


In [5]:
results = service.users().messages().list(
    userId = 'me',
    maxResults = 2
).execute()

messages = results.get('messages', [])
print(messages)

[{'id': '19bc9c52fc39b0e9', 'threadId': '19bc9c52fc39b0e9'}, {'id': '19bc9a524d0af894', 'threadId': '19bc9a524d0af894'}]


In [6]:
def get_message(service, msg_id):
    return service.users().messages().get(
        userId = 'me',
        id = msg_id,
        format = 'full'
    ).execute()

msg = get_message(service, messages[0]['id'])
print(msg.keys())
print(msg['snippet'])

dict_keys(['id', 'threadId', 'labelIds', 'snippet', 'payload', 'sizeEstimate', 'historyId', 'internalDate'])
Hello forsignins043, I&#39;m glad to invite you to take part in Codeforces Round 1074 (Div. 4). It starts on Sunday, January, 18, 2026 14:35 (UTC). The contest duration is 2 hours 15 minutes. The


In [7]:
def extract_email_data(msg):
    headers = msg['payload']['headers']
    data = {}

    for h in headers:
        name = h['name']
        if name in ['From', 'To', 'Subject', 'Date']:
            data[name] = h['value'] 

    parts = msg['payload'].get('parts')
    body = ""

    if parts:
        for part in parts:
            if part['mimeType'] == 'text/plain':
                body = base64.urlsafe_b64decode(
                    part['body']['data']
                ).decode("utf-8", errors="ignore")
    else:
        body = base64.urlsafe_b64decode(
            msg['payload']['body']['data']
        ).decode("utf-8", errors="ignore")

    data["body"] = body
    return data


email_data = extract_email_data(msg)
print(email_data)


{'Date': 'Sat, 17 Jan 2026 04:54:40 +0300 (MSK)', 'From': 'Codeforces@codeforces.com', 'To': 'forsignins043@gmail.com', 'Subject': 'Codeforces Round 1074 (Div. 4)', 'body': "Hello forsignins043,\r\n\r\nI'm glad to invite you to take part in Codeforces Round 1074 (Div. 4). It starts on Sunday, January, 18, 2026 14:35 (UTC). The contest duration is 2 hours 15 minutes. The allowed programming languages are C/C++, Python, Java, Kotlin, Rust, Go, C#, F#, Pascal/Delphi, JavaScript, D, Haskell, OCaml, Perl, PHP, Ruby, Scala.\r\n\r\nDiv. 4 rounds are designed for participants with the rating below 1400 (and for unrated participants/newbies).\r\n\r\nRegister Now →  The format of the event will be identical to Div. 3 and Educational Rounds. In short:\r\n\r\n* Not only real problems, but also exercises can be used.\r\n* Our focus is on delivering good training problems, not on how novel they are.\r\n* Often the formal text of the statements.\r\n* Rated for participant with the rating below 1400.\

In [8]:
for m in messages:
    msg = get_message(service, m['id'])
    email = extract_email_data(msg)

    print("\n----Mail----\n")
    print("From:", email.get("From"))
    print("Subject:", email.get("Subject"))
    print("Body:", email["body"][:500])


----Mail----

From: Codeforces@codeforces.com
Subject: Codeforces Round 1074 (Div. 4)
Body: Hello forsignins043,

I'm glad to invite you to take part in Codeforces Round 1074 (Div. 4). It starts on Sunday, January, 18, 2026 14:35 (UTC). The contest duration is 2 hours 15 minutes. The allowed programming languages are C/C++, Python, Java, Kotlin, Rust, Go, C#, F#, Pascal/Delphi, JavaScript, D, Haskell, OCaml, Perl, PHP, Ruby, Scala.

Div. 4 rounds are designed for participants with the rating below 1400 (and for unrated participants/newbies).

Register Now →  The format of the eve

----Mail----

From: Medium Daily Digest <noreply@medium.com>
Subject: The Privacy Tax: Why Data is the Price of Survival in the Digital South | consumator
Body: Stories for sanapalasanjay
@forsignins043 (https://medium.com/@forsignins043?source=email-783c4607f195-1768598419398-digest.reader-------------------------c1dba2a0_5936_4cb8_99eb_f42a43114d60)
·Become a member (https://medium.com/plans?source=email-7